# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute time series data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
IS = PowerSystems.IS

InfrastructureSystems

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using Cbc # mip solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)
using Ipopt # solver that supports duals
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
base_dir = PowerSystems.download(PowerSystems.TestData; branch = "master");
pm_data = PowerSystems.PowerModelsData(joinpath(base_dir, "matpower", "case5_re_uc.m"))

FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts", "7day")

tsp_da = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_da_7day.json"),
)
tsp_rt = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_rt_7day.json"),
)
tsp_agc = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_agc_7day.json"),
)

sys_DA = System(pm_data)
reserves = [
    VariableReserve{ReserveUp}("REG1", true, 5.0, 0.1),
    VariableReserve{ReserveUp}("REG2", true, 5.0, 0.06),
    VariableReserve{ReserveUp}("REG3", true, 5.0, 0.03),
    VariableReserve{ReserveUp}("REG4", true, 5.0, 0.02),
]
contributing_devices = get_components(Generator, sys_DA)
for r in reserves
    add_service!(sys_DA, r, contributing_devices)
end

add_time_series!(sys_DA, tsp_da)
transform_single_time_series!(sys_DA, 48, Hour(24))

sys_RT = System(pm_data)
add_time_series!(sys_RT, tsp_rt)
transform_single_time_series!(sys_RT, 12, Hour(1))

sys_AGC = System(pm_data)
add_time_series!(sys_AGC, tsp_agc)

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:362
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:362
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:703
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1222
┌ Info: the voltage setpoint on generator 4 does not match the value at bus 4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1779
┌ Info: the voltage setpoint on generator 1 does not match the value at bus 1
└ @ PowerSystems /Users/cbarrows/.julia/packages/Po

## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = ThermalDispatch
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 


### Define the Simulation Sequence

In [6]:
stages_definition = Dict(
    "UC" => Stage(
        GenericOpProblem,
        template_uc,
        sys_DA,
        solver,
        balance_slack_variables = true,
    ),
    "ED" => Stage(
        GenericOpProblem,
        template_ed,
        sys_RT,
        ipopt_solver,
        constraint_duals = [:CopperPlateBalance],
    ),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])

order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 48, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = mkpath(joinpath(".","5-bus-simulation"))
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

BuildStatus.BUILT = 0

### Execute simulation

In [9]:
execute!(sim)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:29
  Step:                  1
  Stage:                 ED
  Simulation Timestamp:  2020-01-01T23:00:00


RunStatus.SUCCESSFUL = 0

## Results
First we can load the result metadata

In [10]:
results = SimulationResults(sim)
uc_results = get_stage_results(results, "UC")
ed_results = get_stage_results(results, "ED");

┌ Info: checking integrity of ./5-bus-simulation/5bus-test/11/data_store/simulation_store.h5
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/utils.jl:386
┌ Info: Loaded time series from storage file existing=system-0f7fbf78-9e9a-40b2-952b-63bfbd45f732_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_QCaHwV
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/i88mL/src/hdf5_time_series_storage.jl:82
┌ Info: Loaded time series from storage file existing=system-7959c483-4622-4000-bd5d-d38bb6ed1f99_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_q2THmU
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/i88mL/src/hdf5_time_series_storage.jl:82


Then we can read and examine the results of interest

In [11]:
prices = read_dual(ed_results, :CopperPlateBalance)

DataStructures.SortedDict{DateTime,DataFrame,Base.Order.ForwardOrdering} with 24 entries:
  DateTime("2020-01-01T00:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T01:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T02:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T03:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T04:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T05:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T06:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T07:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T08:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T09:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T10:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T11:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T12:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T13:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T14:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T15:00:00") => 12×2 DataFrame…
  DateTime("2020-01-01T16:00:00") => 12×2 DataFrame…
  DateTim

or if we want to look at the realized values

In [12]:
read_realized_duals(ed_results)[:CopperPlateBalance]

,CopperPlateBalance
,Float64
1,100000.0
2,100000.0
3,100000.0
4,100000.0
5,100000.0
6,100000.0
7,100000.0
8,100000.0
9,100000.0


*note that in this simulation the prices are all equal to the balance slack
penalty value of $100000/MWh because there is unserved energy in the result*

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*